In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as l
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

%matplotlib inline

In [2]:
# Fashion MNIST Dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = tf.cast(x_train, tf.float32) / 255.
x_test = tf.cast(x_test, tf.float32) / 255.
y_train_cat = keras.utils.to_categorical(y_train, num_classes=10)
y_test_cat =  keras.utils.to_categorical(y_test, num_classes=10)

In [3]:
def create_simple_classifier():
    inputs = l.Input(shape=(28,28,1))
    z = l.Flatten()(inputs)
    z = l.Dense(200, activation='relu')(z)
    z = l.Dense(100, activation='relu')(z)
    output = l.Dense(10, activation='softmax')(z)
    
    return keras.Model(inputs=[inputs], outputs=[output])

In [4]:
tf.random.set_seed(42)
simple_clf = create_simple_classifier()
simple_clf.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(1e-3), metrics=['accuracy'])
simple_clf.fit(x_train, y_train_cat, epochs=1)

1875/1875 [==============================] - 6s 3ms/step - loss: 0.4820 - accuracy: 0.8275


# TensorBoard

In [5]:
# Load the TensorBoard Externsion
%load_ext tensorboard

In [6]:
!rm -rf ./logs/
tensorboard_callback = keras.callbacks.TensorBoard(histogram_freq=1)

In [7]:
tf.random.set_seed(42)
simple_clf = create_simple_classifier()
simple_clf.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(1e-3), metrics=['accuracy'], )
simple_clf.fit(x_train, y_train_cat, epochs=5, callbacks=[tensorboard_callback])

Epoch 1/5
   1/1875 [..............................] - ETA: 0s - loss: 2.3902 - accuracy: 0.0938WARNING:tensorflow:From /Users/amandeepsaini/.pyenv/versions/ml1/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4820 - accuracy: 0.8275
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3598 - accuracy: 0.8680
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3250 - accuracy: 0.8796
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3017 - accuracy: 0.8887
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2854 - accuracy: 0.8931


#### Lets add l1 reg and see how that affects the histograms of the weights

In [9]:
def create_simple_classifier_reg():
    inputs = l.Input(shape=(28,28,1))
    z = l.Flatten()(inputs)
    z = l.Dense(200, activation='relu', 
                kernel_regularizer=keras.regularizers.l1(l1=0.01))(z)
    z = l.Dense(100, activation='relu',
                kernel_regularizer=keras.regularizers.l1(l1=0.01))(z)
    output = l.Dense(10, activation='softmax')(z)
    
    return keras.Model(inputs=[inputs], outputs=[output])

In [10]:
!rm -rf ./logs/
tensorboard_callback = keras.callbacks.TensorBoard(histogram_freq=1)

In [11]:
tf.random.set_seed(42)
simple_clf_reg = create_simple_classifier_reg()
simple_clf_reg.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(1e-3), metrics=['accuracy'], )
simple_clf_reg.fit(x_train, y_train_cat, epochs=5, callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 2.9996 - accuracy: 0.6771
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 1.3147 - accuracy: 0.7336
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 1.2458 - accuracy: 0.7530
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 1.2088 - accuracy: 0.7623
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 1.1841 - accuracy: 0.7645


In [12]:
%tensorboard --logdir logs/

Reusing TensorBoard on port 6006 (pid 92077), started 0:17:01 ago. (Use '!kill 92077' to kill it.)

#### In the histograms tab, we can see that overtime, the historgams start to have concentrated, sharp peaks, and zeros elsewhere. This is desired with the l1 reg, since we are trying to get sparse weights. This kind of behavior does not happen as strongly without l1 reg, so the visualization showed us that the regularization is indeed making an impact

In [22]:
def create_simple_classifier_cnn():
    inputs = l.Input(shape=(28,28,1))
    z = l.Conv2D(16, 3, activation='relu')(inputs)
    z = l.Conv2D(16, 3, activation='relu')(z)
#     z = l.Flatten()(z)
    z = l.GlobalAveragePooling2D()(z)
    output = l.Dense(10, activation='softmax')(z)
    
    return keras.Model(inputs=[inputs], outputs=[output])

In [23]:
!rm -rf ./logs/
tensorboard_callback = keras.callbacks.TensorBoard(histogram_freq=1)

In [24]:
tf.random.set_seed(42)
simple_clf_cnn = create_simple_classifier_cnn()
simple_clf_cnn.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(1e-3), metrics=['accuracy'], )
simple_clf_cnn.fit(x_train, y_train_cat, epochs=5, callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 42s 23ms/step - loss: 1.3874 - accuracy: 0.5079
Epoch 2/5
1875/1875 [==============================] - 48s 25ms/step - loss: 0.9943 - accuracy: 0.6530
Epoch 3/5
1875/1875 [==============================] - 47s 25ms/step - loss: 0.9201 - accuracy: 0.6775
Epoch 4/5
1875/1875 [==============================] - 39s 21ms/step - loss: 0.8781 - accuracy: 0.6945
Epoch 5/5
1875/1875 [==============================] - 42s 22ms/step - loss: 0.8459 - accuracy: 0.7075


In [35]:
tensorboard_callback = keras.callbacks.TensorBoard(histogram_freq=1, log_dir="logs/trial_3")

In [36]:
# tf.random.set_seed(42)
simple_clf = create_simple_classifier()
simple_clf.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(1e-3), metrics=['accuracy'], )
simple_clf.fit(x_train, y_train_cat, epochs=5, callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4825 - accuracy: 0.8267
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3621 - accuracy: 0.8665
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3282 - accuracy: 0.8791
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3046 - accuracy: 0.8874
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2858 - accuracy: 0.8923
